In [1]:
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
#import matplotlib.pyplot as plt
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
import pickle
from sklearn.model_selection import train_test_split
import torch.optim as optim
from helper_functions import train_or_load

In [2]:
#load the shadow model trained in the other python script
device = device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
resnet_shadow =  models.mobilenet_v2(weights=None,num_classes = 200)

resnet_cifar = torch.load("shadow_models/mobilenet_shadow_tinyimage_overtrained.pth",map_location=device)
resnet_shadow.load_state_dict(resnet_cifar)

c:\Users\User-1\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\User-1\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


<All keys matched successfully>

In [3]:
DATA_PATH = 'pickle/tinyimagenet/mobilenetv2/shadow.p'
# Change the DATA_PATH to your local pickle file path


with open(DATA_PATH, "rb") as f:
    dataset = pickle.load(f)


#splitting
#only use train set here
train_data, val_data = train_test_split(dataset, test_size=(1-0.5),shuffle=False)
  
dataloader = torch.utils.data.DataLoader(train_data, batch_size=1, shuffle=False, num_workers=2)
testloader =  torch.utils.data.DataLoader(val_data, batch_size=1, shuffle=True, num_workers=2)

for batch_idx, (img, label) in enumerate(dataloader):
    img = img.to(device)


In [4]:
#generate dataset for attack model training
resnet_shadow.eval()
dataset_attack = []
with torch.no_grad():
    for images, labels in testloader: #need only one
            # Move images and labels to the appropriate device
        images, labels = images.to(device), labels.to(device)
            
            # Forward pass
        logits = resnet_shadow(images)
        
        #take the 3 biggest logist
        top_values = torch.topk(logits, k=3).values
        top_values, indices = torch.sort(top_values, dim=1, descending=True)
        dataset_attack.append([top_values,0])
        
with torch.no_grad():
    for images, labels in dataloader: #need only one
            # Move images and labels to the appropriate device
        images, labels = images.to(device), labels.to(device)
            
            # Forward pass
        logits = resnet_shadow(images)
        
        #take the 3 biggest logist
        
        top_values = torch.topk(logits, k=3).values
        top_values, indices = torch.sort(top_values, dim=1, descending=True)

        dataset_attack.append([top_values,1])
        



In [5]:
standardized_data_list = dataset_attack

# # Convert all tensors to the same dtype first
# tensors = [data[0].float() for data in dataset_attack]  # Ensure all tensors are Float type
# all_data = torch.cat(tensors, dim=0)  # Concatenate all tensors

# # Calculate mean and std
# mean = all_data.mean(dim=0)
# std = all_data.std(dim=0)

# # Standardize data in the list
# standardized_data_list = [( (data[0] - mean) / std, data[1] ) for data in dataset_attack]


In [6]:
dataloader_attack = torch.utils.data.DataLoader(standardized_data_list, batch_size=64, shuffle=True, num_workers=2) #shuffled training data

In [7]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(3, 32)
        self.bn1 = nn.BatchNorm1d(32)  # Matches the output of fc1
        self.fc2 = nn.Linear(32, 64)
        self.bn2 = nn.BatchNorm1d(64)  # Matches the output of fc2
        self.fc3 = nn.Linear(64, 32)
        self.bn3 = nn.BatchNorm1d(32)  # Matches the output of fc3
        self.fc4 = nn.Linear(32, 16)
        self.bn4 = nn.BatchNorm1d(16)  # Matches the output of fc4
        self.fc5 = nn.Linear(16, 8)
        self.bn5 = nn.BatchNorm1d(8)   # Matches the output of fc5
        self.fc6 = nn.Linear(8, 1)
        self.dropout = nn.Dropout(0.5)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))
        x = torch.relu(self.bn2(self.fc2(x)))
        x = torch.relu(self.bn3(self.fc3(x)))
        x = torch.relu(self.bn4(self.fc4(x)))
        x = self.dropout(torch.relu(self.bn5(self.fc5(x))))
        x = self.sigmoid(self.fc6(x))
        return x


In [8]:
attack_model = SimpleNN()
criterion = nn.BCELoss()
optimizer = optim.Adam(attack_model.parameters(), lr=0.0001)

In [9]:
#dict =  torch.load('attack_model.pth_3', map_location='cpu')
#model.load_state_dict(dict)

In [10]:
train_or_load(attack_model,dataloader_attack,optimizer,criterion,5)

Epoch: 1
tensor([[1.7859, 1.1660, 1.0927]])
tensor([0.])
Batch 0 with Loss: 0.7742007970809937
tensor([[2.2882, 0.9182, 0.8732]])
tensor([1.])
Batch 50 with Loss: 0.7192357182502747
tensor([[3.7081, 2.1735, 1.9821]])
tensor([1.])
Batch 100 with Loss: 0.7025209665298462
tensor([[3.3852, 2.0995, 2.0257]])
tensor([0.])
Batch 150 with Loss: 0.7233453989028931
tensor([[2.7052, 2.0890, 1.5056]])
tensor([0.])
Batch 200 with Loss: 0.6828556656837463
tensor([[0.9985, 0.3835, 0.3269]])
tensor([1.])
Batch 250 with Loss: 0.7053641080856323
tensor([[0.5660, 0.4249, 0.3271]])
tensor([0.])
Batch 300 with Loss: 0.6885164976119995
tensor([[2.1141, 1.7693, 1.6562]])
tensor([1.])
Batch 350 with Loss: 0.7109758853912354
tensor([[1.5047, 1.0248, 1.0113]])
tensor([1.])
Batch 400 with Loss: 0.7347774505615234
tensor([[0.9441, 0.6927, 0.5244]])
tensor([1.])
Batch 450 with Loss: 0.708190381526947
tensor([[1.6871, 1.6283, 1.5531]])
tensor([0.])
Batch 500 with Loss: 0.7166533470153809
tensor([[1.8333, 1.6350, 1.

In [11]:
torch.save(attack_model.state_dict(), 'attack_models/attack_mobilenet_tinyimage.pth')

<h1>Evaluation</h1>

In [13]:
DATA_PATH = "pickle/tinyimagenet/mobilenetv2/eval.p"

device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

with open(DATA_PATH, "rb") as f:
    dataset = pickle.load(f)
# Convert all tensors to the same dtype first

eval_dataloader = torch.utils.data.DataLoader(
    dataset, batch_size=1 , shuffle=False, num_workers=2)
# #splitting
# for batch_idx, (img, label, membership) in enumerate(dataloader):
#     img = img.to(device)


In [14]:
# Load target model
resnet_target_model =  models.mobilenet_v2(weights=None,num_classes = 200)
check = torch.load("models/mobilenetv2_tinyimagenet.pth", map_location=device)
resnet_target_model.load_state_dict(check["net"])
resnet_target_model.eval()

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [15]:
#get posteriors from target
target_dataset_eval = []
with torch.no_grad():
    for images,_, member in eval_dataloader: #need only one
            # Move images and labels to the appropriate device
        images, labels = images.to(device), labels.to(device)
            
            # Forward pass
        logits = resnet_target_model(images)
        
        #take the 3 biggest logist
        
        top_values = torch.topk(logits, k=3).values #order poseri
        sorted_tensor, indices = torch.sort(top_values, dim=1,descending=True)
        target_dataset_eval.append([sorted_tensor, member.item()])
        


In [16]:
# #onvert all tensors to the same dtype first
# tensors = [data[0].float() for data in dataset_eval]  # Ensure all tensors are Float type
# all_data = torch.cat(tensors, dim=0)  # Concatenate all tensors

# # Calculate mean and std
# mean = all_data.mean(dim=0)
# std = all_data.std(dim=0)

# # Standardize data in the list
# dataset = [( (data[0] - mean) / std, data[1] ) for data in dataset_eval]

In [17]:
dataloader_eval = torch.utils.data.DataLoader(target_dataset_eval, batch_size=1 , shuffle=False, num_workers=2)

In [18]:
def evaluate(model, dataloader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient computation
        for data in dataloader:
            inputs = data[0]
            
            labels = data[1].float()
            labels = labels.float()
            optimizer.zero_grad()

            outputs = model(inputs.squeeze(dim=1))
            predicted = torch.round(outputs)  # Round the outputs to 0 or 1
            print(predicted,labels)

            total += labels.size(0)  # Increment the total count by batch size
            correct += (predicted == labels).sum().item()  # Count correct predictions

    accuracy = correct / total
    return accuracy
accuracy = evaluate(attack_model, dataloader_eval)

print(f'Accuracy: {accuracy:.2f}')

C:\Users\User-1\AppData\Local\Temp\ipykernel_17384\3255356309.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels).float()


tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tenso